In [1]:
import yfinance as yf
import backtrader as bt

from datetime import datetime, timedelta

In [2]:
from modules import utils

In [3]:
# %matplotlib inline

# Pulling Stock Data

In [4]:
start_year = 2013
start_month = 1
start_day = 1

end_year = 2013
end_month = 1
end_day = 3


start_year = None
start_month = None
start_day = None

end_year = None
end_month = None
end_day = None


# Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
interval = '30m'

data = utils.pull_stock(
    symbol='AAPL',
    interval=interval,
    start_year=start_year,
    start_month=start_month,
    start_day=start_day,
    end_year=end_year,
    end_month=end_month,
    end_day=end_day,
)

data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-09-27 09:30:00-04:00,152.899994,154.720001,152.850006,153.919998,153.919998,12947636
2022-09-27 10:00:00-04:00,153.919998,154.039993,152.500000,152.789902,152.789902,7338970
2022-09-27 10:30:00-04:00,152.789505,153.720001,152.729996,153.476196,153.476196,5329738
2022-09-27 11:00:00-04:00,153.460007,153.570007,151.570007,151.755005,151.755005,5474965
2022-09-27 11:30:00-04:00,151.755005,152.389999,151.529999,151.574997,151.574997,4913498
...,...,...,...,...,...,...
2022-10-06 14:00:00-04:00,146.439301,146.439301,145.720001,145.884003,145.884003,2732957
2022-10-06 14:30:00-04:00,145.880005,146.160004,145.270004,145.690002,145.690002,3246123
2022-10-06 15:00:00-04:00,145.679993,145.880005,145.250000,145.869995,145.869995,4068525


# Example

### Create Engine

In [5]:
cerebro = bt.Cerebro()  # create a "Cerebro" engine instance

### Strategy

#### Define Strategy 

In [6]:
class SmaCross(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position




#### Add Strategy

In [7]:
cerebro.addstrategy(SmaCross)  # Add the trading strategy

0

### Data

#### Example Data
Doesn't work

In [8]:
# Create a data feed
# example_data = bt.feeds.YahooFinanceData(dataname='AAPL',
#                                  fromdate=datetime(2011, 1, 1),
#                                  todate=datetime(2012, 12, 31))

# cerebro.adddata(example_data)  # Add the data feed

#### yFinance Data

In [9]:
data_simple = yf.download('AAPL', start='2010-01-01')
pd_data = bt.feeds.PandasData(dataname=data_simple)

cerebro.adddata(pd_data)  # Add the data feed

[*********************100%***********************]  1 of 1 completed


### Run And Plot

In [10]:
cerebro.run()  # run it all

In [11]:
cerebro.plot()  # and plot it with a single comman

<IPython.core.display.Javascript object>

[[<Figure size 432x288 with 5 Axes>]]

# Strategies

## IBS

In [12]:
th_high_IBS = 0.9
th_low_IBS = 0.1
ma_len = 200

class IbsSolo(Strategy):
    def init(self):
        close = self.data.Close
        self.ma = self.I(SMA, close, ma_len)
    
    def next(self):
        close = self.data.Close
        low = self.data.Low
        high = self.data.High
        
        IBS = (close - low)/(high - low)
        
        if IBS > th_high_IBS:
            self.sell()
        elif IBS < th_low_IBS and self.ma*1.05 > close:
            self.buy()
    
    
class IbsSma(Strategy):
    def init(self):
        pass
    
    def next(self):
        close = self.data.Close
        low = self.data.Low
        high = self.data.High
        
        IBS = (close - low)/(high - low)
        
        if IBS > th_high_IBS:
            self.sell()
        elif IBS < th_low_IBS:
            self.buy()

NameError: name 'Strategy' is not defined

In [ ]:
bt = Backtest(GOOG, IbsSolo, commission=.002,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

In [ ]:
stats